# RT Patient Scheduling for CT and LINAC- MIP Model

**Fan Jia**

**Date: 2021-02-25**

In [108]:
import numpy as np
import pandas as pd
#import plotly.express as px

import gurobipy as gp
from gurobipy import Model, GRB, quicksum, max_
from random import randint, seed

seed(0)


In [109]:
df_partial = pd.DataFrame(columns=['MRN', 'Unit', 'FracsRemain','TxDur'])
df_record = pd.DataFrame(columns=['MRN','CreatedDate','WaitTime','CTUnit','LinacUnit'])

## Introduction

### 1. Sets
       
   * $j \in J$: A patient, $j$, in a set of patients, $J$: 
       * Ex. $J = \left \{1,2,3,\ldots,j  \right \}$
       
   * $|J|$ = n: number of patients waiting to be booked for their appointments
   
   * Each patient $j$ has a site group, a category (i.e. emergency level), a prescrition (number of LINAC treatments)
   
   * Each patient $j$ has to booked for 1 CT appointment and a certain number of LINAC appts for consecutive days as prescribed
   
   * Depending on patient site group, the patient can be booked on 2 CTs and 2 LINACs that are capable
   
   * Each patient has a certain number of days in between CT and LINAC first appointment - predicted

   * $c \in C$: a CT, $c$, in a set of CTs, $C$: 
       * Ex. $C = \left \{1,2,3,\ldots,c  \right \}$
       
   * $l \in L$: a LINAC machine, $l$, in a set of LINACs that are capable to treat this patient based on sitegroup
 
   * $L = \left \{1,2,3\right\}$: is the set of LINAC machines in PMCC
   
   
   * $M_c$ : Set of CT simulators
   
   * $M_l$ : Set of LINACs units
   * $M_{lj}$ : Set of LINAC units that are capable to treat patient $j$
       

### 2. Parameters

   * $C_{j}$ : Cosultation date for patient $j$
   * $w_j$   : Weight assigned to patient $j$ based on category/urgency
   * $p_{j}$ : Days needed in between CT and first LINAC appoitment
   * $S_{j}$ : Number of treatment sessions prescribed for patient $j$
   * $n_{j}$ : Number of days in between two LINAC appointment. (Assuming to be 1 for now)
   
   
   * Machine Capacities
   * $m_d^{l}$: LINAC $l$ capacity on day $d$ (Available working hours)
   * $m_d^{c}$: CT $c$ capacity on day $d$ (Available working hours)
   
   
   * Appointment Durations
   * $D_{cj}$: Duration of CT appointment for patient $j$
   * $D_{lj}^s$: Duration of $s^{th}$ treatment session for patient $j$
   

In [110]:
# daily machine available hours (randomly generated)
CT_hours=pd.read_excel('CThours_endOfWarmup.xlsx') #"ins_machine_hours.xlsx", sheet_name='CT')
LINAC_hours=pd.read_excel("Linachours_endOfWarmup.xlsx") #"ins_machine_hours.xlsx", sheet_name='LINAC')

#site_team_machine = pd.read_excel("Sitegroup_machines.xlsx", sheet_name='Sheet2')

In [111]:
# read sample input instance
instance_all=pd.read_excel("ins_201910_202002_MIP.xlsx")
#instance = instance[instance['Team']==1]

In [112]:
all_dates = instance_all['CreatedDate'].unique()
all_dates.sort()
all_dates

array(['2019-10-01T00:00:00.000000000', '2019-10-02T00:00:00.000000000',
       '2019-10-03T00:00:00.000000000', '2019-10-04T00:00:00.000000000',
       '2019-10-07T00:00:00.000000000', '2019-10-08T00:00:00.000000000',
       '2019-10-09T00:00:00.000000000', '2019-10-10T00:00:00.000000000',
       '2019-10-11T00:00:00.000000000', '2019-10-13T00:00:00.000000000',
       '2019-10-15T00:00:00.000000000', '2019-10-16T00:00:00.000000000',
       '2019-10-17T00:00:00.000000000', '2019-10-18T00:00:00.000000000',
       '2019-10-19T00:00:00.000000000', '2019-10-21T00:00:00.000000000',
       '2019-10-22T00:00:00.000000000', '2019-10-23T00:00:00.000000000',
       '2019-10-24T00:00:00.000000000', '2019-10-25T00:00:00.000000000',
       '2019-10-28T00:00:00.000000000', '2019-10-29T00:00:00.000000000',
       '2019-10-30T00:00:00.000000000', '2019-10-31T00:00:00.000000000',
       '2019-11-01T00:00:00.000000000', '2019-11-04T00:00:00.000000000',
       '2019-11-05T00:00:00.000000000', '2019-11-06

In [155]:
# for one day
instance=instance_all[instance_all['CreatedDate']==pd.to_datetime('2020-01-01')]
instance.head()

,MRN,CreatedDate,CreatedDay,SiteGroup,Intent,Category_x,SimApptDuration,TltDose,TxFracs,TxApptDuration,...,WaitTime(weekday),Contour,Plan,PretrtDays,priority,Sitegroup,Team_y,CT,linacs,targetWait
150,907979,2020-01-01,0,BREAST,Palliative,Urgent 2,30,800,1,30,...,21,0,0,0,10,BREAST,2,3,"5, 6, 7, 8, 11, 14",3


In [156]:
# Sets
# set of patients:
J = [j for j in range(len(instance['MRN']))]
num_patients=len(J)

# set of CT simulators:
CTs = [c for c in [2,3,4]] 
# set of LINAC machines:
LINACs = [l for l in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17]]

# planning horizon: 10 days + 1 artificial day
H = [d for d in range(1,22)]    # d is from 0 or from 1..?... one
# The cosultation day in the horizon
C = [j for j in instance['CreatedDay']] #used to calculate wait time


p=[] # pretreatment days
D1=[] # CT appointment durations
D2=[] # linac appointment durations
cap_linacs=[]
cap_CTs=[]
F=[]
pr=[]
for index, row in instance.iterrows():
    j=index
    s_j = int(row['TxFracs'])
    F.append(s_j) #set of number of fracs for patients
    
    D1_j = row['SimApptDuration']
    D1.append(D1_j) # set of CT duration
    D2_j = row['TxApptDuration']
    D2.append(D2_j) # set of linac durations
    p_j = row['PretrtDays']
    p.append(p_j) # set of req. pre-treatment days
    
    linac_j = row['linacs']
    list_linacs = list(map(int, list(linac_j.split(','))))
    cap_linacs.append(list_linacs) # set of lists of capable linac machines
    
    CT_j = row['CT']
    if isinstance(CT_j, str):
        list_CTs = list(map(int, list(CT_j.split(','))))
        cap_CTs.append(list_CTs) # set of assigned CTs
    else:
        cap_CTs.append([CT_j])
      
    pr_j = row['priority']
    pr.append(pr_j)
    #sessions=[s for s in range(1,s_j+1)]
    #sub_B = [(j,d,l,s) for d in H for s in sessions for l in LINACs]
 

### 3. Create Gurobi Model

In [157]:
with gp.Env(empty=True) as env:
    env.setParam('LogToConsole', 0)
    env.setParam('OutputFlag', 0)
    env.start()
    with gp.Model(env=env) as model:
        model = gp.Model("RT_Patient_Scheduling")

Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1


#Create a new model
model = gp.Model("RT_Patient_Scheduling")
model.setParam('TimeLimit', 300) # seconds
model.Params.LogFile='LogFile'
#model.setParam("OutputFlag", 0)
model.setParam('LogToConsole',0)
#model.Params.LogToConsole = 0


### 4. Decision Variables

   * Binary Variables: 
   
   $x_{jdc}$
   $\begin{equation}
  =\left\{
  \begin{array}{@{}ll@{}}
    1, & \text{if patient j is booked for CT simulation on day d on CT c} \\
    0, & \text{otherwise}
  \end{array}\right.
\end{equation} $

    Schdule the first fraction first, only $y_{jdl}$ for now

   $y_{jdl}$
   $\begin{equation}
  =\left\{
  \begin{array}{@{}ll@{}}
    1, & \text{if patient j is booked for first treatment on day d on LINAC machine l} \\ 
    0, & \text{otherwise}
  \end{array}\right.
\end{equation} $

   $\bar y_{jdl}$
   $\begin{equation}
  =\left\{
  \begin{array}{@{}ll@{}}
    1, & \text{if patient j is booked on day d} \\ 
    0, & \text{otherwise}
  \end{array}\right.
\end{equation} $

   $z_{j}$
   $\begin{equation}
  =\left\{
  \begin{array}{@{}ll@{}}
    1, & \text{if patient j waits more than 14 days for the first treatment} \\ 
    0, & \text{otherwise}
  \end{array}\right.
\end{equation} $
   
   $L_{jl}$
   $\begin{equation}
  =\left\{
  \begin{array}{@{}ll@{}}
    1, & \text{if patient j is assigned to linac machine l} \\ 
    0, & \text{otherwise}
  \end{array}\right.
\end{equation} $
   
   
   
   * $d$ = index of the day in the planning horizon
   
   * $d_{1j}$ = index of the day CT is booked for patient $j$
   
   * $d_{2j}$ = index of the day first treatment is booked for patient $j$


In [158]:
# create a list of tuples containing every iteration of j,d,c
x_tuple = [(j,d,c) for j in J for d in H for c in CTs]
#len(x_tuple)


In [159]:
# create a list of tuples containing iterations of j,d,l,s_j
y_tuple=[]
p_list=[]
for j in J:
    p_list = [(j,d,l,s) for d in H for l in LINACs for s in range(0,F[j])]
    y_tuple+=p_list
#y_tuple=[(j,d,l) for j in J for d in H for l in LINACs]
#len(y_tuple)

In [160]:
# create a list of tuples containing iterations of j,l
L_tuple = [(j,l) for j in J for l in LINACs]
#len(L_tuple)

In [161]:
#Create decision variables
x = model.addVars(x_tuple, vtype=GRB.BINARY, name="x_jdc")
y = model.addVars(y_tuple, vtype=GRB.BINARY, name="y_jd1")
#y_bar = model.addVars(y_bar_tuple, vtype=GRB.BINARY, name="y_bar_jdl")
z = model.addVars(num_patients, vtype=GRB.BINARY, name="z_j")

L = model.addVars(L_tuple, vtype=GRB.BINARY, name="L_jl")

d1 = model.addVars(num_patients, vtype=GRB.INTEGER, name="d_1j")
d2 = model.addVars(num_patients, vtype=GRB.INTEGER, name="d_2j")

O = model.addVars(num_patients, vtype=GRB.INTEGER, name="O_j")
U = model.addVars(num_patients, vtype=GRB.INTEGER, name="U_j")
W = model.addVars(num_patients, vtype=GRB.INTEGER, name="waitTime_j")


### The planning horizon consists of 20 normal weekdays and an artificial day representing the future planning horizons.
#### on this "day 21", multiple tx could be booked

In [162]:
# objective
#model.setObjective(quicksum(z[j]*O[j]*pr[j] + W[j] for j in J), GRB.MINIMIZE)
model.setObjective(quicksum(O[j]*pr[j] for j in J), GRB.MINIMIZE)
                                  #..+ W[j].............................................

In [163]:
# 1. dummy constraints for target wait time
model.addConstrs((W[j]==d2[j]-C[j] for j in J), name='wait_time_calc')
model.addConstrs((10 == W[j]-O[j]+U[j] for j in J), name='overage_underage')


{0: <gurobi.Constr *Awaiting Model Update*>}

In [164]:
# 2. define z variables: if z=1, WaitTime > 14 days; if z=0, WaitTime <= 14 days
# big_M=99999
# small_m = 0.00001
# model.addConstrs((W[j]-14 <= z[j] * big_M for j in J),name='bigM')
# model.addConstrs((W[j]-14 > z[j] * small_m for j in J), name='smallm')

# strict inequality is not supported in gurobi.....


In [165]:
# if z[j]==0, WaitTime[j]<=14
model.addConstrs(((z[j]==0)>>(W[j]<=10) for j in J), name='wait_time_z_var')
model.addConstrs(((z[j]==1)>>(W[j]>=11) for j in J), name='wait_time_z_var')


{0: <gurobi.GenConstr *Awaiting Model Update*>}

In [166]:
# 3. Book first treatment appt on d2 for patient j
#model.addConstrs((quicksum(y[j,d,l] for l in LINACs) * d <= d2[j] for j in J for d in H), name='book_linac')
# Book CT appointment on d1_j
#model.addConstrs((quicksum(x[j,d,c] for c in CTs) * d <= d1[j] for j in J for d in H),name='book_CT')


In [167]:
# if v=0, patient j is not scheduled on day d; if v=1, patient j's 1st fraction is scheduled on day d 
v_tuple = [(j,d) for j in J for d in H]  
v = model.addVars(v_tuple, vtype=GRB.BINARY, name="v_jd")

model.addConstrs((v[j,d]==0)>>(quicksum(y[j,d,l,0] for l in LINACs)<=0) for j in J for d in H)
model.addConstrs((v[j,d]==1)>>(d2[j]==d) for j in J for d in H)

v2 = model.addVars(v_tuple, vtype=GRB.BINARY, name="v2_jd") # same for the CT appointment
model.addConstrs((v2[j,d]==0)>>(quicksum(x[j,d,c] for c in CTs)<=0) for j in J for d in H)
model.addConstrs((v2[j,d]==1)>>(d1[j]==d) for j in J for d in H)


{(0, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 6): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 7): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 8): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 9): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 10): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 11): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 12): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 13): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 14): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 15): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 16): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 17): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 18): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 19): <gurobi.GenConstr *Awaiting

In [168]:
# 4. Sufficient time inbetween CT Sim and LINAC first treatment
model.addConstrs((d2[j]-d1[j] >= (p[j])*quicksum(x[j,d,c] for c in CTs for d in H) for j in J),
                 name='pretreat_duration')
# change H[:-1] to H.........................................................................................

{0: <gurobi.Constr *Awaiting Model Update*>}

In [169]:
# 5. Only one CT appointment should be book for each patient
model.addConstrs(
    (quicksum(x[j,d,c] for c in CTs for d in H) == 1 for j in J), name='one_CT')

# 6. Only one first fraction should be booked for each patient
for j in J:
    model.addConstrs(
        (quicksum(y[j,d,l,0] for l in LINACs for d in H) == 1 for j in J), name='one_first_linac')

In [170]:
# 7. Each patient is assigned to one linac machine
model.addConstrs((L[j,l]==quicksum(y[j,d,l,0] for d in H) for j in J for l in LINACs), name='define_L')
model.addConstrs(
    (quicksum(L[j,l] for l in LINACs) == 1 for j in J), name='one_linac_per_patient')


{0: <gurobi.Constr *Awaiting Model Update*>}

In [171]:
# 8. All treatment fractions are booked on the same linac machine patient j is assigned to
for j in J:
    model.addConstrs(
        (quicksum(y[j,d,l,s] for d in H) <= L[j,l] for s in range(0,F[j]) for l in LINACs), name='on_same_linac')

In [172]:
# 9. Patient j should only be treated by the linac that are capable
for j in J:
    model.addConstrs(
        (y[j,d,l,s] == 0 for s in range(0,F[j]) for l in LINACs if l not in cap_linacs[j] for d in H), 
                                                                                        name='capable_linac')


In [173]:
# schedule on the CT that is assigned
for j in J:
    model.addConstrs(
        (x[j,d,c] == 0 for c in CTs if c not in cap_CTs[j] for d in H), name = 'capable_CT')
    
#model.addConstrs(
 #   (x[j,d,c] == 0 for j in J for d in H for c in CTs if c not in cap_CTs[j]), name='assign_CT') # not in cap_linacs[j]


## Consecutive tx ... in progress

In [174]:
# 10. consecutive day treatment

B = model.addVars(num_patients, vtype=GRB.BINARY, name="B_j")
model.addConstrs((B[j]==0)>>(11-d2[j]-F[j] >= 0) for j in J) # all tx schedule in H
model.addConstrs((B[j]==1)>>(11-d2[j]-F[j] <= -1) for j in J) # only part of tx is schedule in H



f=model.addVars(num_patients, vtype=GRB.INTEGER, name="f_j")
model.addConstrs((B[j]==0)>>(f[j]==(F[j]-1)) for j in J)
model.addConstrs((B[j]==0)>>(quicksum(y_bar[j,d,l] for l in LINACs for d in H)==F[j]-1) for j in J)
model.addConstrs((B[j]==0)>>(f[j]==len(H)-d2[j]) for j in J)
model.addConstrs((B[j]==1)>>(quicksum(y_bar[j,d,l] for l in LINACs for d in H)==len(H)-d2[j]) for j in J)

In [175]:
# 10.1 total number of frac booked is less or equal to the num prescribed
for j in J:
    model.addConstr(
        (quicksum(y[j,d,l,s] for d in H for l in LINACs for s in range(0,F[j])) <= F[j] ), name="num_frac"
    )
                    # == F[j]

In [176]:
# one treatment per day
#for j in J:
 #   model.addConstrs((quicksum(y[j,d,l,s] for l in LINACs for s in range(0,F[j]))<=1 for j in J for d in H), 
                                         #                                                name='one_tx_per_day')

In [177]:
# 10.2 treatments on consecutive days                
for j in J:
    model.addConstrs(
        (
            y[j,d+1,l,s+1] == y[j,d,l,s] for l in LINACs for s in range(0,F[j]-1) for d in H[:-1]
         ), name='consecutive_days')

In [178]:
# 11. machine working hours
# for CTs
model.addConstrs(                                  #.........added -30 mins...........
        (quicksum(D1[j]*x[j,d,c] for j in J) <= CT_hours[CT_hours['day']==d][c] for d in H for c in CTs),
        name='CT_hours')
model.addConstrs(
        (quicksum(D1[j]*x[j,d,c] for j in J) >= 0 for d in H for c in CTs),
        name='CT_hours_pos')


{(1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 4): <gurobi.Constr *Awaiting Model Update*>,
 (5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (5, 3): <gurobi.Constr *Awaiting Model Update*>,
 (5, 4): <gurobi.Constr *Awaiting Model Update*>,
 (6, 2): <gurobi.Constr *Awaiting Model Update*>,
 (6, 3): <gurobi.Constr *Awaiting Model Update*>,
 (6, 4): <gurobi.Constr *Awaiting Model Update*>,
 (7, 2): <gurobi.Constr *Awaiting Model Update*>,
 (7, 3): <gurobi.Constr *Awaiting Model Update*>,


In [179]:
# for linacs, both first sessions and the rest of the sessions???...
# for all patients on this machine on this date <= capacity[d,l]

model.addConstrs(
        (quicksum(D2[j]*y[j,d,l,s] for j in J for s in range(0,F[j])) <= LINAC_hours[LINAC_hours['day']==d][l]
                    for d in H for l in LINACs), name = 'linac_hours')
model.addConstrs(
        (quicksum(D2[j]*y[j,d,l,s] for j in J for s in range(0,F[j])) >= 0 for d in H for l in LINACs),
        name='linac_hours_pos')

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Upd

In [180]:
# domains:
model.addConstrs((d1[j] >= 0 for j in J), name='domain_d1')
model.addConstrs((d2[j] >= d1[j] for j in J), name='domain_d2')
model.addConstrs((O[j] >= 0 for j in J), name='domain_O')
model.addConstrs((U[j] >= 0 for j in J), name='domain_U')
model.addConstrs((W[j] >= 0 for j in J), name='domain_W')


{0: <gurobi.Constr *Awaiting Model Update*>}

### 7. Optimize Gurobi Model

In [181]:
model.optimize()


Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 1094 rows, 463 columns and 2518 nonzeros
Model fingerprint: 0x1f7ef3b7
Model has 86 general constraints
Variable types: 0 continuous, 463 integer (458 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 1054 rows and 388 columns
Presolve time: 0.01s
Presolved: 40 rows, 75 columns, 112 nonzeros
Presolved model has 37 SOS constraint(s)
Variable types: 0 continuous, 75 integer (37 binary)

Root relaxation: objective 0.000000e+00, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    2          -    0.00000      -     -    0s
H    0     0                       0.0000000    0.00000  0.00%     -    0s
     0     0    0.00000    0  

In [182]:
status = model.status
if status == GRB.UNBOUNDED:
    print('The model cannot be solved because it is unbounded')
    #sys.exit(0)
if status == GRB.OPTIMAL:
    print('The optimal objective is %g' % model.objVal)
    #for v in model.getVars():
        #print('%s %g' % (v.varName, v.x))       
    model.printAttr('X')
    #sys.exit(0)
    
if status != GRB.INF_OR_UNBD and status != GRB.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)
    #sys.exit(0)
    
if status == GRB.INFEASIBLE:
    # do IIS
    print('The model is infeasible; computing IIS')
    model.computeIIS()
    if model.IISMinimal:
        print('IIS is minimal\n')
    else:
        print('IIS is not minimal\n')
    print('\nThe following constraint(s) cannot be satisfied:')
    for c in model.getConstrs():
        if c.IISConstr:
            print('%s' % c.constrName)
    
    
    

The optimal objective is 0

    Variable            X 
-------------------------
x_jdc[0,4,3]            1 
y_jd1[0,4,8,0]            1 
   L_jl[0,8]            1 
     d_1j[0]            4 
     d_2j[0]            4 
      U_j[0]            6 
waitTime_j[0]            4 
   v_jd[0,4]            1 
  v2_jd[0,4]            1 
Optimization was stopped with status 2


In [183]:
# Store solution
if model.status == GRB.OPTIMAL:
    sol_c = model.getAttr('X', x)
    sol_l = model.getAttr('X', y)
    sol_d2 = model.getAttr('X', d2)
    sol_w = model.getAttr('X', W)
    
    # non-zero variables
    x_jdc = [i for i in sol_c if sol_c[i] > 0.5]
    y_jdls = [i for i in sol_l if sol_l[i] > 0.5]
    #waitTime = [i for i in ]
    
#sol_w


In [184]:
# testing
CT_hours=pd.read_excel('CThours_endOfWarmup.xlsx') #"ins_machine_hours.xlsx", sheet_name='CT')
LINAC_hours=pd.read_excel("Linachours_endOfWarmup.xlsx") #"ins_machine_hours.xlsx", sheet_name='LINAC')


In [185]:
CT_hours


,day,2,3,4
0,1,5,15,17
1,2,5,5,2
2,3,5,10,2
3,4,80,160,17
4,5,100,120,60
5,6,100,120,60
6,7,100,120,60
7,8,100,120,60
8,9,100,120,60
9,10,100,120,60


In [186]:
### update machine hours - CT 
CT_hours_copy = CT_hours
CT_hours_copy['Day']=CT_hours_copy['day']
CT_hours_copy.set_index('Day', inplace=True)

for i in x_jdc:
    p = i[0]
    d = i[1]
    ct = i[2]   
    # ct appt duration
    time_c = instance.iloc[p]['SimApptDuration']    
    current = CT_hours_copy.loc[d][ct]
    new = current - time_c    
    CT_hours_copy.loc[d, ct] = new

### roll forward for the next day - CTs
# dropping first and last row
CT_hours_copy = CT_hours_copy.iloc[1:-1]

#reset the day number
CT_hours_copy['day']=CT_hours_copy['day']-1

# add two new rows: one for a new day, the other for the arbituary day
df2 = pd.DataFrame([[20, 170, 235, 137],[21,1000,1000,1000]], columns=['day',2,3,4])
CT_hours_updated=CT_hours_copy.append(df2, ignore_index=True)
CT_hours_updated.index = CT_hours_updated.index + 1 # resetting the index

CT_hours_updated


<ipython-input-186-954e6181e829>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CT_hours_copy['day']=CT_hours_copy['day']-1


,day,2,3,4
1,1,5,5,2
2,2,5,10,2
3,3,80,130,17
4,4,100,120,60
5,5,100,120,60
6,6,100,120,60
7,7,100,120,60
8,8,100,120,60
9,9,100,120,60
10,10,100,120,60


In [187]:
CT_hours_updated

,day,2,3,4
1,1,5,5,2
2,2,5,10,2
3,3,80,130,17
4,4,100,120,60
5,5,100,120,60
6,6,100,120,60
7,7,100,120,60
8,8,100,120,60
9,9,100,120,60
10,10,100,120,60


In [146]:
CT_hours_copy = CT_hours
CT_hours_copy.set_index('day')
current = CT_hours_copy.loc[20][2]
current


100

In [188]:
LINAC_hours


,day,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17
0,1,67,307,77,37,112,147,142,137,297,232,47,117,52,67,137,87
1,2,62,307,52,52,117,72,142,142,202,187,47,102,87,97,62,42
2,3,17,302,37,22,132,57,97,127,172,167,27,87,27,112,42,42
3,4,17,347,37,52,142,42,127,97,127,202,27,52,47,152,87,42
4,5,17,347,17,67,127,27,97,82,157,202,27,27,92,122,102,27
5,6,32,332,32,52,142,27,62,97,202,177,27,27,32,167,87,72
6,7,17,362,42,22,127,72,67,97,237,162,57,82,47,227,22,57
7,8,47,362,42,22,37,82,67,92,237,162,57,67,97,292,42,42
8,9,92,362,62,22,37,107,82,77,252,187,57,102,82,322,87,42
9,10,107,347,62,52,67,122,97,102,252,127,87,147,22,352,57,27


In [189]:
### update machine hours - Linacs
linac_hours_copy = LINAC_hours
linac_hours_copy['Day']=linac_hours_copy['day']
linac_hours_copy.set_index('Day', inplace=True)

for i in y_jdls:
    p = i[0]
    d = i[1]
    linac = i[2] 
    
    # Tx appt duration
    time_l = instance.iloc[p]['TxApptDuration'] 
    current = linac_hours_copy.loc[d][linac]
    new = current - time_l    
    linac_hours_copy.loc[d, linac] = new + 15 #??????

    
### roll forward for the next day - linacs
# dropping first and last row
linac_hours_copy = linac_hours_copy.iloc[1:-1]

# reset the day number
linac_hours_copy['day']=linac_hours_copy['day']-1

# add two new rows: one for a new day, the other for the arbituary day
day20 = [300]*16
day21 = [1000]*16
df2 = pd.DataFrame([[20]+day20,
                    [21]+day21], columns=['day',1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17])

linac_hours_updated = linac_hours_copy.append(df2, ignore_index=True)
#linac_hours_copy.index = linac_hours_copy.index + 1 # resetting the index
linac_hours_updated.index = linac_hours_updated.index + 1 # resetting the index
linac_hours_updated


<ipython-input-189-9408d6822439>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linac_hours_copy['day']=linac_hours_copy['day']-1


,day,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17
1,1,62,307,52,52,117,72,142,142,202,187,47,102,87,97,62,42
2,2,17,302,37,22,132,57,97,127,172,167,27,87,27,112,42,42
3,3,17,347,37,52,142,42,127,82,127,202,27,52,47,152,87,42
4,4,17,347,17,67,127,27,97,82,157,202,27,27,92,122,102,27
5,5,32,332,32,52,142,27,62,97,202,177,27,27,32,167,87,72
6,6,17,362,42,22,127,72,67,97,237,162,57,82,47,227,22,57
7,7,47,362,42,22,37,82,67,92,237,162,57,67,97,292,42,42
8,8,92,362,62,22,37,107,82,77,252,187,57,102,82,322,87,42
9,9,107,347,62,52,67,122,97,102,252,127,87,147,22,352,57,27
10,10,107,347,62,67,47,107,52,117,282,157,87,147,77,352,147,27


In [190]:
### Record Booked Patients
#df_record = pd.DataFrame(columns=['MRN','CreatedDate','WaitTime','CTUnit','LinacUnit'])

list_record=[]

for i in y_jdls:
    p=i[0]
    d=i[1]
    l=i[2]
    s=i[3]
    MRN = instance.iloc[p]['MRN']
    createdDate = instance.iloc[p]['CreatedDate']
    wait = sol_d2[p]
    ct = instance.iloc[p]['CT'] 
    linac = l
    if s == 0: # record the first treatment, if first tx is booked the patient is booked
        list_record.append([MRN, createdDate, wait, ct, linac])
        
df_record = df_record.append(pd.DataFrame(list_record, columns=df_record.columns))

In [191]:
# patients who are partially booked (have appt on arbituary day 21)
#df_partial = pd.DataFrame(columns=['MRN', 'Unit', 'FracsRemain','TxDur'])

list_partial = []
for i in y_jdls:
    p=i[0]
    d=i[1]
    if d == 21:
        bookedFracs = i[3]-1
        remainingFracs = instance.iloc[p]['TxFracs'] - bookedFracs
        unit = i[2] # all fracs booked on the same unit?
        MRN = instance.iloc[p]['MRN']
        TxDur = instance.iloc[p]['TxApptDuration']
        list_partial.append([MRN, unit, remainingFracs, TxDur])
        
df_partial = df_partial.append(pd.DataFrame(list_partial, columns=df_partial.columns))
df_partial = df_partial.reset_index()
df_partial.drop(columns=['index'], inplace = True)

In [192]:
df_partial


,MRN,Unit,FracsRemain,TxDur
0,362342,17,18,25
1,2089526,10,18,30
2,3418056,12,9,45
3,4333316,16,18,50
4,4680119,6,4,30
5,4705168,12,19,30
6,6311890,15,17,30


In [193]:
### book remaining sessions by updating linac hours on the new day 20
# every time the machine hours is rolled forward, day 20 is newly added
# for partially booked patients: on each new day 20,

for index, row in df_partial.iterrows():
    # machine (linac) hours: -1*TxApptDur ( day 20 )
    # df_partial: FracsRemain -1
    # df_partial: drop row if FracsRemain == 0
    unit = row['Unit']
    MRN = row['MRN']
    TxDur = row['TxDur']
    linac_hours_updated.loc[19, unit] -= TxDur # this row is day 20
    df_partial.loc[index, 'FracsRemain'] -= 1 # reduce the remaining fracs by 1
    if row['FracsRemain']==0: # drop from partial booked dataframe
        df_partial.drop(index, inplace = True)
        
    

In [194]:
# use updated linac and CT hours
CT_hours = CT_hours_updated
LINAC_hours = linac_hours_updated